In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from datetime import datetime

from __future__ import print_function
import pickle

In [2]:
url = 'https://www.the-numbers.com/movie/budgets/all/301'

user_agent = {'User-agent': 'Mozilla/5.0'}

response  = requests.get(url, headers = user_agent)

page = response.text

soup = BeautifulSoup(page,"lxml")

In [3]:
table = soup.find('table')
table

<table>
<tr><th> </th><th>Release<br/>Date</th><th>Movie</th><th>Production<br/>Budget</th><th>Domestic<br/>Gross</th><th>Worldwide<br/>Gross</th></tr>
<tr><td class="data">301</td>
<td><a href="/box-office-chart/daily/1999/08/27">Aug 27, 1999</a></td>
<td><b><a href="/movie/13th-Warrior-The#tab=summary">The 13th Warrior</a></b></td>
<td class="data"> $125,000,000</td>
<td class="data"> $32,698,899</td>
<td class="data"> $61,698,899</td>
</tr>
<tr><td class="data">302</td>
<td><a href="/box-office-chart/daily/2000/11/17">Nov 17, 2000</a></td>
<td><b><a href="/movie/How-the-Grinch-Stole-Christmas-(2000)#tab=summary">How the Grinch Stole Christmas</a></b></td>
<td class="data"> $123,000,000</td>
<td class="data"> $260,348,825</td>
<td class="data"> $345,445,403</td>
</tr>
<tr><td class="data">303</td>
<td><a href="/box-office-chart/daily/2000/05/24">May 24, 2000</a></td>
<td><b><a href="/movie/Mission-Impossible-2#tab=summary">Mission: Impossible 2</a></b></td>
<td class="data"> $120,000

In [4]:
rows = [row for row in table.find_all('tr')]

In [5]:
movies = {}

for row in rows[1:]:
    items = row.find_all('td')
    link = row.find('a')
    title, url = link.text, link['href']
    movies[title] = [url] + [i.text for i in items]
    
movies

{'Aug 27, 1999': ['/box-office-chart/daily/1999/08/27',
  '301',
  'Aug 27, 1999',
  'The 13th Warrior',
  '\xa0$125,000,000',
  '\xa0$32,698,899',
  '\xa0$61,698,899'],
 'Nov 17, 2000': ['/box-office-chart/daily/2000/11/17',
  '302',
  'Nov 17, 2000',
  'How the Grinch Stole Christmas',
  '\xa0$123,000,000',
  '\xa0$260,348,825',
  '\xa0$345,445,403'],
 'May 24, 2000': ['/box-office-chart/daily/2000/05/24',
  '303',
  'May 24, 2000',
  'Mission: Impossible 2',
  '\xa0$120,000,000',
  '\xa0$215,409,889',
  '\xa0$549,588,516'],
 'Jun 30, 2000': ['/box-office-chart/daily/2000/06/30',
  '304',
  'Jun 30, 2000',
  'The Perfect Storm',
  '\xa0$120,000,000',
  '\xa0$182,618,434',
  '\xa0$328,711,434'],
 'Jul 20, 2016': ['/box-office-chart/daily/2016/07/20',
  '305',
  'Jul 20, 2016',
  'Jason Bourne',
  '\xa0$120,000,000',
  '\xa0$162,192,920',
  '\xa0$416,168,316'],
 'Jun 15, 2007': ['/box-office-chart/daily/2007/06/15',
  '306',
  'Jun 15, 2007',
  'Fantastic Four: Rise of the Silver Surfe

In [6]:
budget4 = pd.DataFrame(movies).T  #transpose
budget4.columns = ['link_stub','rank_all_movies','release','title', 
                    'budget', 'domestic_gross','worldwide_gross']

budget4.shape

(98, 7)

In [7]:
budget4['budget'] = budget4['budget'].str.replace('$', '')
budget4['budget'] = budget4['budget'].str.replace(',', '')
budget4['budget'] = budget4['budget'].astype(int)

In [8]:
budget4.head()

,link_stub,rank_all_movies,release,title,budget,domestic_gross,worldwide_gross
"Aug 27, 1999",/box-office-chart/daily/1999/08/27,301,"Aug 27, 1999",The 13th Warrior,125000000,"$32,698,899","$61,698,899"
"Nov 17, 2000",/box-office-chart/daily/2000/11/17,302,"Nov 17, 2000",How the Grinch Stole Christmas,123000000,"$260,348,825","$345,445,403"
"May 24, 2000",/box-office-chart/daily/2000/05/24,303,"May 24, 2000",Mission: Impossible 2,120000000,"$215,409,889","$549,588,516"
"Jun 30, 2000",/box-office-chart/daily/2000/06/30,304,"Jun 30, 2000",The Perfect Storm,120000000,"$182,618,434","$328,711,434"
"Jul 20, 2016",/box-office-chart/daily/2016/07/20,305,"Jul 20, 2016",Jason Bourne,120000000,"$162,192,920","$416,168,316"


In [9]:
budget4.to_csv('budget4.csv')